In [1]:
import pandas as pd

In [2]:
df_master = pd.read_pickle('pickle/master.pickle')
df_teamsplits = pd.read_pickle('pickle/team_splits_mi.pickle')

In [3]:
df_master.sample(5)

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
playerID,,,,,,,,,
higgich01,Chris,Higgins,C,1983.0,6.0,2.0,USA,NY,Smithtown
scandma01,Marco,Scandella,D,1990.0,2.0,23.0,Canada,QC,Montreal
stpiema01,Martin,St. Pierre,C,1983.0,8.0,11.0,Canada,ON,Embrun
nikolan01,Andrei,Nikolishin,C,1973.0,3.0,25.0,Russia,NaN,Vorkuta
rydermi01,Michael,Ryder,R,1980.0,3.0,31.0,Canada,NL,St. John's


In [4]:
df_master = df_master.assign(birthDate = pd.to_datetime({
    'year': df_master.birthYear,
    'month': df_master.birthMon,
    'day': df_master.birthDay
}))

df_master = df_master.drop(columns=['birthYear', 'birthMon', 'birthDay'])
df_master.head()

,firstName,lastName,pos,birthCountry,birthState,birthCity,birthDate
playerID,,,,,,,
aaltoan01,Antti,Aalto,C,Finland,NaN,Lappeenranta,1975-03-04
abdelju01,Justin,Abdelkader,L,USA,MI,Muskegon,1987-02-25
abidra01,Ramzi,Abid,L,Canada,QC,Montreal,1980-03-24
actonke01,Keith,Acton,C,Canada,ON,Stouffville,1958-04-15
adamlu01,Luke,Adam,L,Canada,NL,St. John's,1990-06-18


In [5]:
df_master.birthDate.head()

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
abidra01    1980-03-24
actonke01   1958-04-15
adamlu01    1990-06-18
Name: birthDate, dtype: datetime64[ns]

In [8]:
ts = df_master.birthDate[0]
ts

Timestamp('1975-03-04 00:00:00')

In [9]:
print(u"{}, {} {} {}".format(ts.day_name(), ts.day,
                             ts.month_name(),ts.year))

Tuesday, 4 March 1975


In [10]:
tsz = ts.tz_localize('America/Toronto')
tsz

Timestamp('1975-03-04 00:00:00-0500', tz='America/Toronto')

In [11]:
tsz.tz_convert('Europe/London')

Timestamp('1975-03-04 05:00:00+0000', tz='Europe/London')

In [13]:
years = df_master.birthDate.dt.year
years.head()

playerID
aaltoan01    1975
abdelju01    1987
abidra01     1980
actonke01    1958
adamlu01     1990
Name: birthDate, dtype: int64

In [14]:
strings = df_master.birthDate.dt.strftime("%Y-%d-%Y-%a")
strings.head()

playerID
aaltoan01    1975-04-1975-Tue
abdelju01    1987-25-1987-Wed
abidra01     1980-24-1980-Mon
actonke01    1958-15-1958-Tue
adamlu01     1990-18-1990-Mon
Name: birthDate, dtype: object

In [15]:
strings = strings.str.replace("-", "xx")
strings.head()

playerID
aaltoan01    1975xx04xx1975xxTue
abdelju01    1987xx25xx1987xxWed
abidra01     1980xx24xx1980xxMon
actonke01    1958xx15xx1958xxTue
adamlu01     1990xx18xx1990xxMon
Name: birthDate, dtype: object

In [16]:
try:
    pd.to_datetime(strings)
except Exception as e:
    print(e)

('Unknown string format:', '1975xx04xx1975xxTue')


In [21]:
birth_dates = pd.Series(df_master.index, index=df_master.birthDate)
birth_dates.head()

birthDate
1975-03-04    aaltoan01
1987-02-25    abdelju01
1980-03-24     abidra01
1958-04-15    actonke01
1990-06-18     adamlu01
Name: playerID, dtype: object

In [22]:
birth_dates.index

DatetimeIndex(['1975-03-04', '1987-02-25', '1980-03-24', '1958-04-15',
               '1990-06-18', '1977-03-20', '1977-04-26', '1960-05-31',
               '1963-08-15', '1974-10-08',
               ...
               '1987-09-01', '1963-05-08', '1987-03-03', '1987-02-14',
               '1970-07-22', '1978-06-16', '1987-09-01', '1992-01-16',
               '1954-04-16', '1978-01-21'],
              dtype='datetime64[ns]', name='birthDate', length=4085, freq=None)

In [23]:
birth_dates = birth_dates.sort_index()
birth_dates.head()

birthDate
1948-07-10    reschgl01
1950-01-27    bublaji01
1950-07-12    melocgi01
1950-11-13    perregi01
1950-12-12    bouchda01
Name: playerID, dtype: object

In [24]:
born_1980 = birth_dates['1980']
born_1980.head()

birthDate
1980-01-02    zalesmi01
1980-01-10    stanara01
1980-01-13    bouckty01
1980-01-13     ruppmi01
1980-01-15    papinju01
Name: playerID, dtype: object

In [25]:
birth_dates['1980-07-20':'1980-07-31']

birthDate
1980-07-20    arsende01
1980-07-22    kalindm01
1980-07-24    printda01
1980-07-24    jillsje01
1980-07-31    fischji01
Name: playerID, dtype: object

In [26]:
diff = birth_dates.index[1] - birth_dates.index[0]
diff

Timedelta('566 days 00:00:00')

In [27]:
diff.components

Components(days=566, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0, nanoseconds=0)

In [28]:
diff = diff + '5H' - '10M'
diff

Timedelta('566 days 04:50:00')

In [29]:
birth_dates = birth_dates.to_period(freq="D")
birth_dates.index

PeriodIndex(['1948-07-10', '1950-01-27', '1950-07-12', '1950-11-13',
             '1950-12-12', '1950-12-12', '1951-04-17', '1951-05-05',
             '1951-06-02', '1951-08-03',
             ...
             '1992-07-31', '1992-09-08', '1992-10-05', '1992-10-27',
             '1992-11-12', '1992-11-23', '1992-12-07', '1993-03-15',
             '1993-04-12', '1993-04-18'],
            dtype='period[D]', name='birthDate', length=4085, freq='D')

In [30]:
birth_dates.index[0]

Period('1948-07-10', 'D')

In [31]:
start = birth_dates.index[0].start_time
start

Timestamp('1948-07-10 00:00:00')

In [33]:
end = birth_dates.index[0].end_time
start < (start + pd.Timedelta('5H')) < end

True

In [34]:
birth_dates.index.year

Int64Index([1948, 1950, 1950, 1950, 1950, 1950, 1951, 1951, 1951, 1951,
            ...
            1992, 1992, 1992, 1992, 1992, 1992, 1992, 1993, 1993, 1993],
           dtype='int64', name='birthDate', length=4085)

In [35]:
birth_dates.resample('1M')

PeriodIndexResampler [freq=<MonthEnd>, axis=0, closed=right, label=right, convention=start, base=0]

In [36]:
birth_dates.resample('1M').count().head()

birthDate
1948-07    1
1948-08    0
1948-09    0
1948-10    0
1948-11    0
Freq: M, Name: playerID, dtype: int64

In [37]:
birth_dates.shift(1, freq="D").head()

birthDate
1948-07-11    reschgl01
1950-01-28    bublaji01
1950-07-13    melocgi01
1950-11-14    perregi01
1950-12-13    bouchda01
Freq: D, Name: playerID, dtype: object

In [38]:
df_teamsplits.head()

L  OL    T    W
name          year month                   
Boston Bruins 1980 Apr    2.0 NaN  0.0  1.0
                   Dec    6.0 NaN  1.0  6.0
                   Feb    4.0 NaN  2.0  6.0
                   Jan    4.0 NaN  1.0  9.0
                   Mar    4.0 NaN  3.0  8.0

In [39]:
df_teamsplits = df_teamsplits.reset_index()
df_teamsplits.head()

,name,year,month,L,OL,T,W
0,Boston Bruins,1980,Apr,2.0,NaN,0.0,1.0
1,Boston Bruins,1980,Dec,6.0,NaN,1.0,6.0
2,Boston Bruins,1980,Feb,4.0,NaN,2.0,6.0
3,Boston Bruins,1980,Jan,4.0,NaN,1.0,9.0
4,Boston Bruins,1980,Mar,4.0,NaN,3.0,8.0


In [41]:
string_date = df_teamsplits.apply(lambda x:
                                pd.Period("{}-{}".format(x.year, x.month)),
                                axis=1)
string_date[0]

Period('1980-04', 'M')

In [42]:
df_teamsplits = df_teamsplits.assign(month = string_date)
df_teamsplits = df_teamsplits.drop("year", axis=1)
df_teamsplits.head()

,name,month,L,OL,T,W
0,Boston Bruins,1980-04,2.0,NaN,0.0,1.0
1,Boston Bruins,1980-12,6.0,NaN,1.0,6.0
2,Boston Bruins,1980-02,4.0,NaN,2.0,6.0
3,Boston Bruins,1980-01,4.0,NaN,1.0,9.0
4,Boston Bruins,1980-03,4.0,NaN,3.0,8.0


In [43]:
df_teamsplits = df_teamsplits.set_index(["month"])
df_teamsplits.head()

,name,L,OL,T,W
month,,,,,
1980-04,Boston Bruins,2.0,NaN,0.0,1.0
1980-12,Boston Bruins,6.0,NaN,1.0,6.0
1980-02,Boston Bruins,4.0,NaN,2.0,6.0
1980-01,Boston Bruins,4.0,NaN,1.0,9.0
1980-03,Boston Bruins,4.0,NaN,3.0,8.0


In [44]:
quarter_sums = df_teamsplits.groupby("name").resample("Q").sum()
quarter_sums.head()

L   OL    T     W
name          month                       
Anaheim Ducks 2006Q1  13.0  6.0  0.0  19.0
              2006Q2   0.0  2.0  0.0   1.0
              2006Q3   0.0  0.0  0.0   0.0
              2006Q4   7.0  6.0  0.0  28.0
              2007Q1  10.0  3.0  0.0  26.0

In [46]:
df_teamsplits.to_pickle('pickle/teamsplits_periods.pickle')
birth_dates.to_pickle('pickle/birth_dates.pickle')